In [ ]:
!pip install -q timm
!pip install -q git+https://github.com/PyFstat/PyFstat@python37

In [ ]:
# import wandb
import os, gc, re
from PIL import Image
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import numpy as np, pandas as pd
from collections import defaultdict
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

import skimage
from skimage import io

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

class CFG:
    wandb=False
    competition='G2Net'
    model='inception_v4'
    apex=False
    max_grad_norm=1.36
    seed=13
    positive_rate=0.5
    signal_low=0.2
    signal_high=0.1
    folds=10
    lr=0.00056
    dropout=0.25
    epochs=3
    gaussian_noise=2.
    one_cycle_pct_start=0.1
    one_cycle=True
    batch=32
    hog=True

In [ ]:
def get_transforms():
    return torchvision.transforms.Compose([
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(mean=0.5, std=0.1)
        ])

def get_transforms_01():
    return torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.RandomChoice([
            augment_time_mask,
            augment_freq_mask,
            torchvision.transforms.Lambda(lambda x: x)
        ], p = [0.3, 0.3, 0.7]),
        torchvision.transforms.Normalize(mean=0.5, std=0.1)
        ])

def get_transforms_02():
    return torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.RandomChoice([
            augment_time_mask,
            augment_freq_mask,
            torchvision.transforms.Lambda(lambda x: x)
        ], p = [0.5, 0.5, 0.5]),
        torchvision.transforms.Normalize(mean=0.5, std=0.1)
        ])

def get_transforms_03():
    return torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.RandomChoice([
            FlipWave(),
            torchvision.transforms.Lambda(lambda x: x)
        ], p = [0.5, 0.5]),
        torchvision.transforms.Normalize(mean=0.5, std=0.1)
        ])


class G2Net_Dataset(nn.Dataset):
    def __init__(
        self,
        df_noise: pd.DataFrame,
        df_signal: pd.DataFrame,
        positive_rate: float = CFG.positive_rate,
        gaussian_noise: float = CFG.gaussian_noise,
        signal_low: float = CFG.signal_low,
        signal_high: float = CFG.signal_high
        is_train: bool = False
    ) -> None:
        self.df_noise = df_noise
        self.df_signal = df_signal
        self.positive_rate = positive_rate
        self.gaussian_noise = gaussian_noise
        self.signal_low = signal_low
        self.signal_high = signal_high
        self.transforms = get_transforms()
        self.is_train = is_train

    def __len__(self):
        return len(self.df_signal)

    def gen_sample(self, signal, noise, signal_strength):
        noise = np.array(Image.open(noise))
        if signal:
            signal = np.array(Image.open(signal))
            noise = noise + signal_strength * signal
        if self.is_train and self.gaussian_noise > 0:
            noise = noise + np.random.randn(*noise.shape) * GAUSSIAN_NOISE 
        noise = np.clip(noise, 0, 255).astype(np.uint8)
        return self.transforms(noise)

    def __getitem__(self, index):
        noise_files = self.df_noise.sample().files.values[0]        
        sig_files = [None, None]
        label = 0.
        if np.random.random() < self.positive_rate:
            sig_files = self.df_signal.sample().files.values[0]
            label = 1.
        signal_strength = np.random.uniform(self.signal_low, self.signal_high)                    
        return np.concatenate(
            [self.gen_sample(sig, noise, signal_strength) for sig, noise in zip(sig_files, noise_files)], axis=0
        ), label

In [ ]:
def net_hog_features(img: np.array, dim: int = 257) ->np.array:    
    img = np.transpose(img.cpu().numpy(), (1, 2, 0))
    bins = np.linspace(0, 1, dim)
    fd = skimage.feature.hog(
        img, orientations=8, pixels_per_cell=(16, 16),
        cells_per_block=(3, 3), visualize=False, multichannel=True
    )
    hist = np.histogram(fd, bins=bins)
    return hist[0]


# -> CONV/FC -> BatchNorm -> ReLu(or other activation) -> Dropout -> CONV/FC ->
class Net(nn.Module):
    def __init__(
        self,
        name_model: str = CFG.model,
        dim: int = 256,
        hog: bool = CFG.hog
    ):
        super().__init__()
        self.model = timm.create_model(
            name_model,
            in_chans=2,
            pretrained=True 
        )
        self.dim = dim
        self.hog = hog
        if self.hog:
            self.s = nn.Linear(1000+self.dim, 1)
        else:
            self.s = nn.Linear(1000, 1)
        # print('Take version: ', self.s, 'dim: ', self.dim)

    def forward(self, x): 
        x1  = self.model(x)
        if not self.hog:
            return self.s(x1)
        tmp = []
        for j in x:
            tmp.append(
                torch.tensor(net_hog_features(j, self.dim + 1), dtype=torch.float, 
            ).reshape(1, -1).to(DEVICE))
        xx =  torch.cat(tmp, axis = 0)
        xx = nn.functional.normalize(xx, p=2.0, dim = 1)
        x1 = nn.functional.normalize(x1, p=2.0, dim = 1) 
        x3 = torch.cat((x1, xx), axis = 1)   
        return self.s(x3)


def train(
    model: nn.Module,
    loader: torch.utils.data.dataloader,
    optimizer: Optional[torch.optim.Optimizer],
    scheduler: Optional[torch.optim.lr_scheduler.LambdaLR],
    epoch: int
) -> None:
    model.train()
    pbar = tqdm(
        loader,
        desc=f"Model Train, epoch: {epoch+1} ",
        total=len(loader),
        mininterval= len(loader)//20
    )
    scaler = torch.cuda.amp.GradScaler(enabled=CFG.apex)
    for X, y in pbar:
        optimizer.zero_grad()
        with torch.autocast(enabled=CFG.apex):
            y_ = model(X.to(DEVICE))
            loss = torch.nn.functional.binary_cross_entropy_with_logits(
                y_.squeeze(), y.to(DEVICE)
            )
        norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if CFG.apex:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()
        if scheduler:
            scheduler.step()


@torch.no_grad()
def evaluate(
    model: nn.Module,
    loader: torch.utils.data.dataloader,
    epoch: int
) -> tuple:
    model.to(DEVICE)
    model.eval()        
    pred = []
    target = []
    pbar = tqdm(
        loader,
        desc=f"Valid, epoch: {epoch+1} ",
        total=len(loader),
        mininterval= len(loader)//20
    )
    for X, y in pbar:
        with torch.autocast(enabled=CFG.apex):
            y_ = model(X.to(DEVICE))
        pred.append(y_.cpu().squeeze())
        target.append(y)
    pred = torch.concat(pred)
    target = torch.concat(target)
    loss = torch.nn.functional.binary_cross_entropy_with_logits(
        pred, target, reduction='none'
    ).median().item()
    return roc_auc_score(target, torch.sigmoid(pred)), loss
    
    
def run_training(dim: int, fold: int = 0) -> None:
    kfold = KFold(CFG.folds, shuffle=True, random_state=CFG.seed)
    df_noise_train, df_noise_eval = None, None
    for f, (tr, vl) in enumerate(kfold.split(df_noise)):
        if f == fold:
            tr_noise = df_noise.loc[tr]
            vl_noise = df_noise.loc[vl]
    df_signal_train, df_signal_eval = None, None
    for f, (tr, vl) in enumerate(kfold.split(df_signal)):
        if f == fold:
            tr_signal = df_signal.loc[tr]
            vl_signal = df_signal.loc[vl]
    tr_data = G2Net_Dataset(
        tr_noise,
        tr_signal,
        is_train=True
    )
    vl_data = G2Net_Dataset(
        vl_noise,
        vl_signal
    )
    tr_loader = torch.utils.data.DataLoader(
        tr_data,
        batch_size=CFG.batch,
        num_workers=os.cpu_count(),
        pin_memory=True
    )
    vl_loder = torch.utils.data.DataLoader(
        vl_data,
        batch_size=CFG.batch,
        num_workers=os.cpu_count(),
        pin_memory=True
    )
    # model = timm.create_model(CFG.model, pretrained=True, num_classes=1, in_chans=2, drop_rate=CFG.dropout)
    # print('Load model: ',  model.__class__.__name__)
    model = Net(dim)
    model.to(DEVICE)
    optim = torch.optim.Adam(model.parameters(), lr=CFG.lr)
    scheduler = None
    if CFG.one_cycle::
        scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimizer=optim,
            max_lr=CFG.lr,
            total_steps=int(len(tr_loader) * CFG.epochs),
            pct_start=CFG.one_cycle_pct_start
    )
    max_auc = 0
    for epoch in range(CFG.epochs):
        train(model, tr_loader, optim, scheduler, epoch)
        auc, loss = evaluate(model, vl_loader, epoch)
        if auc > max_auc:
            torch.save(model.state_dict(), f'model-f{fold}.tph')
            max_auc = auc
        print(f'val_loss: {loss}, val_auc: {auc}, val_max_auc: {max_auc}')
    del model, tr_loader, vl_loder, tr_data, vl_data    
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
import optuna
import logging


logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Setup the root logger.
logger.addHandler(logging.FileHandler("optuna_log.log", mode="w"))

optuna.logging.enable_propagation()  # Propagate logs to the root logger.
optuna.logging.disable_default_handler()  # Stop showing logs in sys.stderr.

def objective(trial: optuna.Trial):
    # CELLS_BLOCK = trial.suggest_categorical('CELLS_BLOCK',[(3, 3), (4, 4), (5, 5)])
    # ORIENTATION = trial.suggest_categorical('ORIENTATION',[7,8,9])
    # PIXELS = trial.suggest_categorical('PIXELS',[(14, 14), (16, 16), (20, 20)])
    # POSITIVE_RATE_GS = trial.suggest_categorical('POSITIVE_RATE_GS', [ 0.1, 0.25, 0.5])    
    # NORM_META = trial.suggest_categorical('NORM_META', [True, False])
    # NORM_MODEL = trial.suggest_categorical('NORM_MODEL', [True, False])
    # VERSION = trial.suggest_categorical('VERSION', [
    #         [False, False, True],
    #         [True, False, False],
    #         [False, True, False],
    #         [False, False, False]]
    # )
    # DIM = trial.suggest_categorical('DIM', [8, 14, 16, 64, 96])
    # OUT_DIM = trial.suggest_categorical('OUT_DIM', [1024, 512, 256, 128])
    CFG.lr = trial.suggest_float('LR', 0.0001, 0.005, log=True)
    CFG.dropout = trial.suggest_categorical('DROPOUT', [0., 0.1, 0.25, 0.3, 0.4, 0.5])
    CFG.max_grad_norm = trial.suggest_float('MAX_GRAD_NORM', 1, 20)
    CFG.epochs = int(trial.suggest_float('EPOCHS', 2, 4, step=1.))
    CFG.gaussian_noise = trial.suggest_categorical('GAUSSIAN_NOISE', [0.,1., 2., 3.])
    CFG.one_cycle_pct_start= trial.suggest_categorical('ONE_CYCLE_PCT_START', [0., 0.1, 0.2, 0.3])
    CFG.model = trial.suggest_categorical('MODEL', ['tf_efficientnetv2_l'])
    CFG.one_cycle = trial.suggest_categorical('ONE_CYCLE', [True, False])
    return run_training(MODEL, 16)

print(ONE_CYCLE_PCT_START, GAUSSIAN_NOISE, MAX_GRAD_NORM, DROPOUT, LR)
study = optuna.create_study(direction='maximize')
logger.info("Start optimization.")
study.optimize(objective, n_trials=60)
with open(f"optuna_log.log") as f:
    assert f.readline().startswith("A new study created")
    assert f.readline() == "Start optimization.\n"